# Source operator modeling and characterization
In this notebook we will construct a genetic network to model a Source operator, a simple constitutive expression device, upload the simulated data to Flapjack, and then show how to characterize the operator based on this data.

## Import required packages

In [ ]:
from loica import *
import matplotlib.pyplot as plt
import numpy as np
import getpass

## Make a connection to Flapjack
Note here you should specify which instance of Flapjack you will use, whether it is local or the public instance for example.

In [ ]:
from flapjack import *
#fj = Flapjack(url_base='flapjack.rudge-lab.org:8000')
fj = Flapjack(url_base='localhost:8000')
fj.log_in(username=input('Flapjack username: '), password=getpass.getpass('Password: '))

## Get or create Flapjack objects
To associate with the components of the genetic network and the simulated data with Flapjack we need the Ids of the appropriate objects. Note that if the objects already exist you will be prompted and can simply hit return to use the existing objects.

In [ ]:
study = fj.create('study', name='Loica testing', description='Test study for demonstrating Loica')

In [ ]:
dna = fj.create('dna', name='source')
vector = fj.create('vector', name='source', dnas=dna.id)

In [ ]:
sfp = fj.create('signal', name='SFP', color='green', description='Simulated fluorescent protein')

## Create the network with measurable reporter
First we create a GeneticNetwork object and associate it with a Flapjack Vector (collection of DNA). The connection to Flapjack is optional, but we will use it here to upload data and characterize our components.

In [ ]:
network = GeneticNetwork(vector=vector.id[0])

Next, we create and add a Reporter object to produce a measurable output signal.

In [ ]:
reporter = Reporter(name='SFP', color='green', degradation_rate=0, init_concentration=0, signal_id=sfp.id[0])

In [ ]:
network.add_reporter(reporter)

## Create the Source operator 
The source operator is a simple constitutive gene, which outputs a constant expression rate $\phi = C$. We create the Source operator object, specifying its output as our previously created Reporter, and its expression rate.

In [ ]:
source = Source(output=reporter, rate=10)
network.add_operator(source)

## Draw the GeneticNetwork as a graph
We can now make a visual representation of our GeneticNetwork to check it is wired up correctly.

In [ ]:
plt.figure(figsize=(3,3), dpi=150)
network.draw()

## Simulate the GeneticNetwork
In order to simulate the GeneticNetwork behaviour we need to specify the growth conditions in which it will operate. To do this we create a SimulatedMetabolism object which specifies growth functions.

In [ ]:
def growth_rate(t):
    return gompertz_growth_rate(t, 0.01, 1, 1, 0.5)

def biomass(t):
    return gompertz(t, 0.01, 1, 1, 0.5)

metab = SimulatedMetabolism(biomass, growth_rate)

Now we can create Samples that contain our GeneticNetwork driven by the SimulatedMetabolism. We also need to specify the Media and Strain, in order to link to the Flapjack data model.

In [ ]:
media = fj.create('media', name='loica', description='Simulated loica media')
strain = fj.create('strain', name='loica', description='Loica test strain')

samples = []
# Create 5 replicate samples for simulation
for _ in range(5):
    sample = Sample(genetic_network=network, 
                    metabolism=metab,
                    media=media.id[0],
                    strain=strain.id[0])
    samples.append(sample)

Given our Samples, we can now create an Assay which will simulate an experiment containing them. We need to specify the biomass signal in order to link to the Flapjack data model for later upload. Running the assay will simulate the behaviour of the GeneticNetwork.

In [ ]:
biomass_signal = fj.create('signal', name='SOD', description='Simulated OD', color='black')
assay = Assay(samples, 
              n_measurements=100, 
              interval=0.24,
              name='Loica constitutive expression (no noise)',
              description='Simulated constitutive gene generated by loica',
              biomass_signal_id=biomass_signal.id[0]
             )
assay.run()

Now we can check the results of our simulations.

In [ ]:
m = assay.measurements
fig,ax = plt.subplots(1,1)
m[m.Signal=='SFP'].plot(x='Time', y='Measurement', style='.', ax=ax)

## Upload simulated data to Flapjack

In [ ]:
assay.upload(fj, study=study.id[0])

## Characterize the Source operator from the uploaded data

In [ ]:
source.characterize(
    fj,
    vector=vector.id,
    media=media.id,
    strain=strain.id,
    signal=sfp.id,
    biomass_signal=biomass_signal.id
)

In [ ]:
source.rate